# Build an End-to-End System

This puts together the chain of prompts that you saw throughout the course.

## Setup
#### Load the API key and relevant Python libaries.
In this course, we've provided some code that loads the OpenAI API key for you.

In [7]:
import os
import openai
import sys
sys.path.append('./resources')
import utils

import panel as pn  # GUI
pn.extension()

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [8]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

## System of chained prompts for processing the user query

In [13]:
def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"
    
    # Step 1: Check input to see if it flags the Moderation API or is a prompt injection
    response = openai.Moderation.create(input=user_input)
    moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        print("Step 1: Input flagged by Moderation API.")
        return "Sorry, we cannot process this request."

    if debug: print("Step 1: Input passed moderation check.")
    
    category_and_product_response = utils.find_category_and_product_only(user_input, utils.get_products_and_category())
    #print(print(category_and_product_response)
    # Step 2: Extract the list of products
    category_and_product_list = utils.read_string_to_list(category_and_product_response)
    #print(category_and_product_list)

    if debug: print("Step 2: Extracted list of products.")

    # Step 3: If products are found, look them up
    product_information = utils.generate_output_string(category_and_product_list)
    if debug: print("Step 3: Looked up product information.")

    # Step 4: Answer the user question
    system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug:print("Step 4: Generated response to user question.")
    all_messages = all_messages + messages[1:]

    # Step 5: Put the answer through the Moderation API
    response = openai.Moderation.create(input=final_response)
    moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        if debug: print("Step 5: Response flagged by Moderation API.")
        return "Sorry, we cannot provide this information."

    if debug: print("Step 5: Response passed moderation check.")

    # Step 6: Ask the model if the response answers the initial user query well
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)
    if debug: print("Step 6: Model evaluated the response.")

    # Step 7: If yes, use this answer; if not, say that you will connect the user to a human
    if "Y" in evaluation_response:  # Using "in" instead of "==" to be safer for model output variation (e.g., "Y." or "Yes")
        if debug: print("Step 7: Model approved the response.")
        return final_response, all_messages
    else:
        if debug: print("Step 7: Model disapproved the response.")
        neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        return neg_str, all_messages

user_input = "tell me about the smartx pro phone and the fotosnap camera, the dslr one. Also what tell me about your tvs"
response,_ = process_user_message(user_input,[])
print(response)

Step 1: Input passed moderation check.
Step 2: Extracted list of products.
Step 3: Looked up product information.
Step 4: Generated response to user question.
Step 5: Response passed moderation check.
Step 6: Model evaluated the response.
Step 7: Model approved the response.
Sure! Here's some information about the SmartX ProPhone and the FotoSnap DSLR Camera:

1. SmartX ProPhone:
   - Brand: SmartX
   - Model Number: SX-PP10
   - Features: 6.1-inch display, 128GB storage, 12MP dual camera, 5G connectivity
   - Description: A powerful smartphone with advanced camera features.
   - Price: $899.99
   - Warranty: 1 year

2. FotoSnap DSLR Camera:
   - Brand: FotoSnap
   - Model Number: FS-DSLR200
   - Features: 24.2MP sensor, 1080p video, 3-inch LCD, interchangeable lenses
   - Description: Capture stunning photos and videos with this versatile DSLR camera.
   - Price: $599.99
   - Warranty: 1 year

Now, could you please let me know which specific TV models you are interested in?


### Function that collects user and assistant messages over time

In [14]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=False)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

### Chat with the chatbot!
Note that the system message includes detailed instructions about what the OrderBot should do.

In [17]:
panels = [] # collect display 

context = [ {'role':'system', 'content':"You are Service Assistant"} ]  

inp = pn.widgets.TextInput( placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/panel/viewable.py:298: ParamFutureWarning: Parameter 'object' on <class 'panel.pane.base.PaneBase'> is being given a valid parameter reference <function _param_bind.<locals>.wrapped at 0x131ef6c10> but is implicitly allow_refs=False. In future allow_refs will be enabled by default and the reference <function _param_bind.<locals>.wrapped at 0x131ef6c10> will be resolved to its underlying value None. Please explicitly set allow_ref on the Parameter definition to declare whether references should be resolved or not.
  super().__init__(**params)


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Service Assistant')
    [2] ParamFunction(function, _pane=Str, defer_load=False, height=300, loading_indicator=True)

## Experiment on your own

In [28]:
# helper functions
import json
from collections import defaultdict

def find_category_and_stock_only(user_input):
    delimiter = "****"
    system_message = f"""
    You will be a stock market analyst, \
    the customer's questions will be delimited with
    {delimiter} characters.
    Output a python list of objects, where each object is a string \
    of one of the following categories:
        'Manufacturing & Consumer Goods', 
        'Technology & E-commerce', 
        'Food & Beverages', 
        'Entertainment & Media', 
        'Energy & Resources', 
        'Automotive & Transportation', 
        'Retail & Fashion', 
        'Healthcare & Pharmaceuticals', 
        'Agriculture & Biotechnology'
    Only output the list of objects, with nothing else.
    """
    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)

stocks = [
    {
        "company": "3M",
        "description": "3M, based in Minnesota, may be best known for its Scotch tape and Post-It Notes, but it also produces sand paper, adhesives, medical products, computer screen filters, food safety items, stationery products and many products used in automotive, marine, and aircraft industries.",
        "industry_vertical": "Manufacturing & Consumer Goods",
        "initial_price": 44.28,
        "end_price": 102.47,
        "news_story": "3M Announces Breakthrough in Sustainable Adhesive Technology, Aiming to Reduce Environmental Footprint",
        "symbol": "MMM"
    },
    {
        "company": "Amazon.com",
        "description": "Amazon.com, Inc. is an online retailer in North America and internationally. The company serves consumers through its retail Web sites and focuses on selection, price, and convenience.",
        "industry_vertical": "Technology & E-commerce",
        "initial_price": 89.38,
        "end_price": 315.20,
        "news_story": "Amazon.com Unveils Plan to Launch a Fleet of Delivery Drones, Revolutionizing E-commerce Logistics",
        "symbol": "AMZN"
    },
    {
        "company": "Campbell Soup",
        "description": "Campbell Soup is a worldwide food company, offering condensed and ready-to-serve soups; broth, stocks, and canned poultry; pasta sauces; Mexican sauces; canned pastas, gravies, and beans; juices and beverages; and tomato juices.",
        "industry_vertical": "Food & Beverages",
        "initial_price": 37.0,
        "end_price": 58.15,
        "news_story": "Campbell Soup to Introduce Plant-Based Soup Line, Targeting the Growing Vegan Market",
        "symbol": "CPB"
    },
    {
        "company": "Disney",
        "description": "The Walt Disney Company, founded in 1923, is a worldwide entertainment company, with movies, cable networks, radio networks, movie production, musical recordings and live stage plays.",
        "industry_vertical": "Entertainment & Media",
        "initial_price": 40.68,
        "end_price": 131.50,
        "news_story": "Disney to Launch Virtual Reality Theme Park, Blending Traditional Rides with VR Experiences",
        "symbol": "DIS"
    },
    {
        "company": "Dow Chemical",
        "description": "The Dow Chemical Company manufactures raw materials that go into consumer products and services. These materials include food and pharmaceutical ingredients, electronic displays, and semiconductor packaging.",
        "industry_vertical": "Manufacturing & Consumer Goods",
        "initial_price": 38.83,
        "end_price": 76.22,
        "news_story": "Dow Chemical Announces Major Breakthrough in Biodegradable Plastics, Aiming to Combat Plastic Waste",
        "symbol": "DOW"
    },
    {
        "company": "Exxon Mobil",
        "description": "Exxon Mobil engages in the exploration and production of crude oil and natural gas, and manufacture of petroleum products.",
        "industry_vertical": "Energy & Resources",
        "initial_price": 39.0,
        "end_price": 112.30,
        "news_story": "Exxon Mobil to Invest Heavily in Renewable Energy, Shifting Focus from Fossil Fuels",
        "symbol": "XOM"
    },
    {
        "company": "Ford",
        "description": "Ford Motor Co. develops, manufactures, sells and services vehicles and parts worldwide. Ford sells cars and trucks primarily under the Ford and Lincoln brands.",
        "industry_vertical": "Automotive & Transportation",
        "initial_price": 27.34,
        "end_price": 47.89,
        "news_story": "Ford Unveils New Self-Driving Car Prototype, Promising a Revolution in Personal Transportation",
        "symbol": "F"
    },
    {
        "company": "The Gap",
        "description": "The Gap, Inc. sells retail clothing, accessories and personal care products globally under the brand names Gap, Old Navy, Banana Republic, and others.",
        "industry_vertical": "Retail & Fashion",
        "initial_price": 46.0,
        "end_price": 33.67,
        "news_story": "The Gap Announces Expansion into Sustainable Fashion, Launching a New Eco-friendly Clothing Line",
        "symbol": "GPS"
    },
    {
        "company": "General Mills",
        "description": "General Mills manufactures and sells consumer foods worldwide. Products include cereals, frozen vegetables, dough, dessert and baking mixes, and frozen pizzas.",
        "industry_vertical": "Food & Beverages",
        "initial_price": 15.59,
        "end_price": 52.74,
        "news_story": "General Mills to Integrate Blockchain for Food Supply Chain Transparency",
        "symbol": "GIS"
    },
    {
        "company": "Hewlett Packard",
        "description": "Hewlett-Packard designs and sells products, technologies, software and IT services to consumers, businesses, government and education sectors worldwide.",
        "industry_vertical": "Technology & E-commerce",
        "initial_price": 66.28,
        "end_price": 89.14,
        "news_story": "Hewlett Packard Launches New Generation of Quantum Computers, Aiming to Transform the Tech Industry",
        "symbol": "HPQ"
    },
    {
        "company": "IBM",
        "description": "IBM is an international IT company. IBM offers infrastructure and technology services, software for business integration and information management, data warehousing, and more.",
        "industry_vertical": "Technology & E-commerce",
        "initial_price": 118.37,
        "end_price": 203.45,
        "news_story": "IBM Develops AI Capable of Predicting Market Trends, Set to Revolutionize Financial Analysis",
        "symbol": "IBM"
    },
    {
        "company": "Johnson & Johnson",
        "description": "Johnson & Johnson develops and manufactures health care products for sale worldwide. J&J products include brands like Johnson’s, Aveeno, Clean & Clear, Neutrogena, and more.",
        "industry_vertical": "Healthcare & Pharmaceuticals",
        "initial_price": 35.13,
        "end_price": 78.90,
        "news_story": "Johnson & Johnson to Launch a New Range of AI-Driven Personal Health Monitoring Devices",
        "symbol": "JNJ"
    },
    {
        "company": "Microsoft",
        "description": "Microsoft develops, manufactures, licenses, and supports a range of software products and services for various computing devices worldwide.",
        "industry_vertical": "Technology & E-commerce",
        "initial_price": 55.72,
        "end_price": 256.80,
        "news_story": "Microsoft Announces New Virtual Reality Operating System, Integrating VR with Daily Computing",
        "symbol": "MSFT"
    },
    {
        "company": "Monsanto",
        "description": "Monsanto provides agricultural products for farmers in the United States and internationally. It operates in two segments, Seeds and Genomics, and Agricultural Productivity.",
        "industry_vertical": "Agriculture & Biotechnology",
        "initial_price": 11.47,
        "end_price": 153.21,
        "news_story": "Monsanto Introduces Drought-Resistant Crops, Aiming to Improve Global Food Security",
        "symbol": "MO"
    },
    {
        "company": "PepsiCo",
        "description": "PepsiCo, Inc. manufactures, markets, and sells various foods, snacks, and carbonated and non-carbonated beverages worldwide.",
        "industry_vertical": "Food & Beverages",
        "initial_price": 34.13,
        "end_price": 105.67,
        "news_story": "PepsiCo to Launch a New Line of Health-Oriented Beverages, Focusing on Wellness and Nutrition",
        "symbol": "PEP"
    },
    {
        "company": "Starbucks",
        "description": "Starbucks Corp. provides specialty coffee and tea beverages, packaged and ground coffee beans, single-serve products, juices and food offerings worldwide.",
        "industry_vertical": "Food & Beverages",
        "initial_price": 6.23,
        "end_price": 48.93,
        "news_story": "Starbucks Expands Into Home Brewing Systems, Launching a New Range of Smart Coffee Machines",
        "symbol": "SBUX"
    },
    {
        "company": "Texas Instruments",
        "description": "Texas Instruments designs and sells semiconductors to electronics designers and manufacturers worldwide. The company has four segments: Analog, Embedded Processing, Wireless and Other.",
        "industry_vertical": "Technology & E-commerce",
        "initial_price": 53.88,
        "end_price": 97.12,
        "news_story": "Texas Instruments Announces Innovative Chip That Can Triple Smartphone Battery Life",
        "symbol": "TXN"
    }
]


In [36]:
# I did not like how the OpenAI example
# had all that code in one function...
# so I split it down into single responsibility functions
# for my use

def check_moderation(user_input, debug):
    response = openai.Moderation.create(input=user_input)
    moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        if debug: print("Input flagged by Moderation API.")
        return False, "Sorry, we cannot process this request."
    if debug: print("Input passed moderation check.")
    return True, ""

def extract_categories_list(user_input, debug):
    category_and_stock_response = find_category_and_stock_only(user_input)
    print(category_and_stock_response)
    if debug: print("Extracted list of stocks.")
    return [category_and_stock_response]

def retrieve_relevant_stocks(categories, stocks_data):
    relevant_stocks = []
    for i in stocks_data:
        if i['industry_vertical'] in categories:
            relevant_stocks.append(i)
    return relevant_stocks

def generate_user_response(user_input, stock_information, all_messages, debug):
    system_message = f"""
    You will be a stock market analyst, \
    you answer the customers questions \
    about the stock market with detail \
    including price information, news, \
    and company information.
    """
    delimiter = "```"
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant stock information:\n{stock_information}"}
    ]
    final_response = get_completion_from_messages(all_messages + messages)
    if debug: print("Generated response to user question.")
    return final_response, all_messages + messages[1:]

def moderate_response(final_response, debug):
    response = openai.Moderation.create(input=final_response)
    moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        if debug: print("Response flagged by Moderation API.")
        return False, "Sorry, we cannot provide this information."
    if debug: print("Response passed moderation check.")
    return True, final_response

def evaluate_response_quality(user_input, final_response, debug):
    delimiter = "```"
    system_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    evaluation_response = get_completion_from_messages([
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_input}
    ])
    if debug: print("Model evaluated the response.")
    return evaluation_response

def finalize_response(evaluation_response, final_response, debug):
    if "Y" in evaluation_response:  # Using "in" to handle variations like "Y." or "Yes"
        if debug: print("Model approved the response.")
        return final_response
    else:
        if debug: print("Model disapproved the response.")
        return "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."

def process_user_message_orchestrated(user_input, all_messages, debug=True):
    passed, response = check_moderation(user_input, debug)
    if not passed: return response, all_messages

    categories = extract_categories_list(user_input, debug)
    retrieve_relevant_stocks(categories, stocks)

    final_response, updated_messages = generate_user_response(user_input, stocks, all_messages, debug)

    passed, response = moderate_response(final_response, debug)
    if not passed: return response, updated_messages

    evaluation_response = evaluate_response_quality(user_input, final_response, debug)
    final_answer = finalize_response(evaluation_response, final_response, debug)

    return final_answer, updated_messages

# Example usage
user_input = "tell me about what happened with PepsiCo today"
response, _ = process_user_message_orchestrated(user_input, [])
print("\n" + response)


Input passed moderation check.
Manufacturing & Consumer Goods
Extracted list of stocks.
Generated response to user question.
Response passed moderation check.
Model evaluated the response.
Model approved the response.

Today, PepsiCo (symbol: PEP) experienced a significant increase in its stock price. The initial price of PepsiCo shares was $34.13, and by the end of the trading day, the price had risen to $105.67.

In terms of news, PepsiCo announced its plans to launch a new line of health-oriented beverages. This new product line will focus on wellness and nutrition, catering to the growing demand for healthier beverage options. This strategic move aligns with PepsiCo's commitment to providing a diverse range of products that meet consumer preferences.

PepsiCo, Inc. is a global food and beverage company that manufactures, markets, and sells various foods, snacks, and carbonated and non-carbonated beverages. The company's portfolio includes popular brands such as Pepsi, Mountain Dew,